<a href="https://colab.research.google.com/github/ns90001/Animorphs/blob/master/StyleGAN_Paintings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/NVlabs/stylegan

In [ ]:
%cd stylegan

In [ ]:
!wget https://github.com/parameter-pollution/stylegan_paintings/releases/download/v0.1/network-snapshot-008040.pkl


In [ ]:
%tensorflow_version 1.x

In [ ]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config

In [ ]:
tflib.init_tf()

In [7]:
model_path = "./network-snapshot-008040.pkl"

In [ ]:
with open(model_path,"rb") as f:
        _G, _D, Gs = pickle.load(f)

In [158]:
latent_vector1 = np.random.randn(1, Gs.input_shape[1])
latent_vector2 = np.random.randn(1, Gs.input_shape[1])

In [161]:
#rnd = np.random.RandomState(1337)

# latent_vector1 = np.random.randn(1, Gs.input_shape[1])
# latent_vector2 = np.random.randn(1, Gs.input_shape[1])

number_of_frames = 480

frame_step = 1.0/number_of_frames
x = 0
for frame_count in range(1,number_of_frames):
    x = x + frame_step
    latent_input = x*latent_vector2 + (1-x)*latent_vector1
    # latent_input = latent_vector1.copy()
    # for i in range(512):
    #     f1 = latent_vector1[0][i]
    #     f2 = latent_vector2[0][i]
    #     if f1 > f2:
    #         tmp = f2
    #         f2 = f1
    #         f1 = tmp
    #     fnew = f1 + (f2-f1)*x
    #     latent_input[0][i] = fnew
    images = Gs.run(latent_input, None, truncation_psi=1, randomize_noise=False, output_transform=fmt)

    # Save image.
    os.makedirs(config.result_dir, exist_ok=True)
    png_filename = os.path.join(config.result_dir, 'animation_'+str(frame_count)+'.png')
    PIL.Image.fromarray(images[0], 'RGB').save(png_filename)

In [ ]:
import os
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

! ffmpeg -framerate 30 -i results/animation_%d.png -vcodec libx264 -pix_fmt yuv420p output.mp4


In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [9]:
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

In [10]:
# change this number to get a different image 
rnd = np.random.RandomState(42)

In [138]:
#rerun this line to get a different image
latent_vector1 = np.random.randn(1, Gs.input_shape[1])

In [ ]:
images = Gs.run(latent_vector1, None, truncation_psi=1, randomize_noise=False, output_transform=fmt)
PIL.Image.fromarray(images[0])